In [1]:
import sys
import dlib
import openface
from skimage import io
import numpy as np
predictor_model = "/home/mckc/Downloads/shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_model)

In [2]:
#Reading the image data into numpy
def rgb2gray(rgb):
    return np.dot(rgb[:,:,:], [0.299, 0.587, 0.114])

In [6]:
def get_landmarks(im,i):
    predictor_model = "/home/mckc/Downloads/shape_predictor_68_face_landmarks.dat"

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_model)
    rects = detector(im, 1)
    
    if len(rects) > 1:
        print 'TooManyFaces',i
        return None
    if len(rects) == 0:
        print 'NoFaces',i
        return None
    coords = np.array([[p.x, p.y] for p in predictor(im, rects[0]).parts()])
    centroid = coords.mean(axis=0)
    return ((coords - centroid )).reshape(1,136)

In [9]:
def load_data():
    import pandas as pd
    import numpy as np
    from PIL import Image
    from termcolor import colored
    
    train = pd.read_csv('/home/mckc/image class//train.csv')
    test = pd.read_csv('/home/mckc/image class//test.csv')
    print 'the training data shape is ',train.shape
    print 'the test data shape is ', test.shape
    
    X_tr = np.zeros((1,136),dtype=np.uint8)
    Y_tr=[]
    for i in range(train.shape[0]):
        image = np.array(Image.open(train.values[i,0]))
        landmarks = get_landmarks(image,train.values[i,0])
        if landmarks is not None:
            X_tr =  np.vstack((X_tr,landmarks))
            Y_tr = np.append(Y_tr,train.values[i,1])
        if i % 50==0:
            print colored((float(i)/train.shape[0]*100 ,' Percentage complete'), 'green')
    
    X_tr = X_tr[1:,:]
    X_ts = np.zeros((1,136),dtype=np.uint8)
    Y_ts=[]
    
    for i in range(test.shape[0]):
        image = np.array(Image.open(test.values[i,0]))
        landmarks = get_landmarks(image,test.values[i,0])
        if landmarks is not None:
            X_ts =  np.vstack((X_ts,landmarks))
            Y_ts = np.append(Y_ts,test.values[i,1])
            
        if i % 50==0:
            print colored((float(i)/test.shape[0]*100 ,' Percentage complete'), 'green')
    X_ts = X_ts[1:,:]
    
    print 'the training file shape',X_tr.shape,Y_tr.shape
    print 'the testing file shape',X_ts.shape,Y_ts.shape
    
    return X_tr,X_ts,Y_tr,Y_ts    

In [10]:
X_tr,X_tst,Y_tr,Y_tst = load_data()

the training data shape is  (114, 2)
the test data shape is  (38, 2)
(0.0, ' Percentage complete')
NoFaces /home/mckc/image class/Mahati_20.jpg
NoFaces /home/mckc/image class/Mahati_14.jpg
NoFaces /home/mckc/image class/Mahati_112.jpg
NoFaces /home/mckc/image class/Nivetha_13.jpg
NoFaces /home/mckc/image class/Mahati_72.jpg
NoFaces /home/mckc/image class/Mahati_75.jpg
TooManyFaces /home/mckc/image class/Nivetha_42.jpg
NoFaces /home/mckc/image class/Mahati_11.jpg
TooManyFaces /home/mckc/image class/Mahati_26.jpg
NoFaces /home/mckc/image class/Mahati_33.jpg
NoFaces /home/mckc/image class/Mahati_51.jpg
NoFaces /home/mckc/image class/Nivetha_34.jpg
NoFaces /home/mckc/image class/Mahati_9.jpg
TooManyFaces /home/mckc/image class/Mahati_41.jpg
NoFaces /home/mckc/image class/Mahati_96.jpg
NoFaces /home/mckc/image class/Mahati_1.jpg
NoFaces /home/mckc/image class/Mahati_82.jpg
NoFaces /home/mckc/image class/Mahati_109.jpg
TooManyFaces /home/mckc/image class/Mahati_40.jpg
NoFaces /home/mckc/imag

IOError: [Errno 2] No such file or directory: '/home/mckc/image class/Mahati_50.jpg'

In [ ]:
map, Y_number = np.unique(Y_tr, return_inverse=True)
Y_test_number = np.unique(Y_tst, return_inverse=True)[1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

clf = LogisticRegression(verbose=0,n_jobs=-1,multi_class='multinomial',solver='lbfgs',max_iter=500,warm_start=True)
clf.fit(X_tr,Y_number)
Y_logictic= clf.predict(X_tst)
Y_log_vales = map[Y_logictic]

print 'Accuracy of the model is ',accuracy_score(Y_tst,Y_log_vales)
confusion_matrix(Y_tst,Y_log_vales)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

recognizer = RandomForestClassifier(500,verbose=0,oob_score=True,n_jobs=-1,max_features=20)
recognizer.fit(X_tr,Y_number)
Y_rf= recognizer.predict(X_tst)
Y_rf_vales = map[Y_rf]

print 'Accuracy of the model is ',accuracy_score(Y_tst,Y_rf_vales)
confusion_matrix(Y_tst,Y_rf_vales)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
from keras.layers import Merge

left_branch = Sequential()
left_branch.add(Dense(1000, input_dim=136,activation='relu'))

right_branch = Sequential()
right_branch.add(Dense(50, input_dim=136,activation='sigmoid'))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(7,activation='softmax'))

final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'],lr=0.0001)
final_model.fit([X_tr,X_tr], Y_Keras,nb_epoch=100, batch_size=1,verbose=1,
          validation_split=0.2, callbacks=[early_stopping])
y_keras = map[final_model.predict_classes([X_tst,X_tst])]

print '/n Accuracy of the model is ',accuracy_score(Y_tst,y_keras)
confusion_matrix(Y_tst,y_keras)